In [2]:
from sage.graphs.graph_coloring import all_graph_colorings
from itertools import combinations

def Mprime(): 
    # makes a variant of Mycielski graph with deleted vertex
    result = graphs.MycielskiGraph(5)
    result.delete_vertex(16)
    result.relabel(range(len(result))) # relabeling of vertices after vertex deletion
    return result 

def get_Is(): 
    # returns independent sets I = get_Is[0] and I' = get_Is[1]
    return ((1, 3, 10, 21), (19, 17, 11, 5))

In [4]:
# sanity check --- testing colors of vertices (1,3,10,21) in all colorings of Mprime() graph
# and connections between I = get_Is[0] and I' = get_Is[1]

M = Mprime()

for i in range(2):
    for e in combinations(get_Is()[i], 2): # sanity check --- if quadruples from get_Is() are independent sets in M
        if M.has_edge(e):
            raise NameError(str(get_Is()[i]) + " is not an independent set! " + str(e))
        
for C in all_graph_colorings(M, 4, vertex_color_dict=True): # iterates over all 4-colorings C of M; color of a vertex v in coloring C is C[v]
    used_colors = set()
    for v in get_Is()[0]:
        used_colors.add(C[v])
    if len(used_colors) < 4:
        raise NameError("not all vertices received different colors!")

print("for v in I =", get_Is()[0], "and w in I' =", get_Is()[1], "there is no edge vw between:")
for v in get_Is()[0]:
    for w in get_Is()[1]:
        if not M.has_edge(v, w):
            print(v, "and", w)

for v in I = (1, 3, 10, 21) and w in I' = (19, 17, 11, 5) there is no edge vw between:
1 and 19
3 and 17
10 and 11
21 and 5


In [3]:
def raw_construction(n): 
    # creates graph with the Mycielski part and n x-type vertices
    result = Mprime()
    for i in range(n):
        result.add_vertex()
    
    for i in range(n):
        for j in (2,3):
            result.add_edge(22+i, get_Is()[0][j])
    return result

def get_color_lists(C): 
    # returns list of pairs (x, L), where x=0 or x=1 indicates
    # which Mycielski quadruple we use for coloring, and L is a list of colors, one of them possibly being C
    # used for coloring of gadgets with vertices ordered a,b,a,b,a,c,c,c
    return ((0, (C,2)), (1, (0,1)), (0, (C,)), (1, (0,1)), (0,(C,3)), (1, (2,3)), (1,(2,3)), (1, (2,3)))

def add_ac_pair(G, C, k, x=-1): 
    # add ac-pair to G
    # C=0 or C=1 is type of a gadget; k=0,1 or 2 is type of ac-pair in an C-type gadget
    # if x = -1, then the pair is not attached to any x-type variable
    
    L = ((C,2), (C,), (C,3))
    a = G.add_vertex()
    c = G.add_vertex()
    G.add_edge(a,c)
    for j in (2,3):
        G.add_edge(c, get_Is()[1][j])
    for j in L[k]:
        G.add_edge(a, get_Is()[0][j])
    if x != -1:
        G.add_edge(c, 22+x)
    
            
def add_gadget(G, n, triple, C): 
    # adds a gadget of type C to G, C=0 or C=1
    # triple = triple of x-type vertices, n - number of x-type vertices in G
    # if len(triple) == 0, then c-type vertices are not adjacent to x-type vertices
    # by default, all b-type vertices are adjacent to all x-type vertices
    new_v = []
    for i in range(8):
        new_v.append(G.add_vertex()) # a-b-a-b-a and three c-type vertices
    for i in range(4):
        G.add_edge(new_v[i], new_v[i+1]) # edges a-b-a-b-a
    for i in range(3):
        G.add_edge(new_v[2*i], new_v[5+i]) # edges a-c
        if len(triple) == 3:
            G.add_edge(new_v[5+i], 22+triple[i]) # edges c-x
    for i in range(n):
        for j in range(2):
            G.add_edge(22+i, new_v[2*j+1]) # edges x-b
            
    recipes = get_color_lists(C)
    for i in range(8):
        for j in recipes[i][1]:
            G.add_edge(new_v[i], get_Is()[recipes[i][0]][j])

def add_clause(G, n, triple): 
    # adds a single clause to G
    # n = number of x-type vertices, triple - triple of x-type vertices
    add_gadget(G, n, triple, 0)
    add_gadget(G, n, triple, 1)

def is_triangle_free(G):
    return G.subgraph_search_count(graphs.CycleGraph(3)) == 0

def is_P19_free(G):
    good = True
    for P in G.subgraph_search_iterator(graphs.PathGraph(19), induced=True):
        good = False
        print("found a long path!", P)
        break
    return good

In [4]:
def G_0(i):
    # constructs a graph G_{0,i} for analyzing induced paths with no x-type vertices
    # assumes i gadgets of type 0 and 5-i gadgets of type 1
    G = raw_construction(0)
    for i in range(i):
        add_gadget(G, 0, [], 0)
    for i in range(5-i):
        add_gadget(G, 0, [], 1)
    return G

In [5]:
for i in range(5):
    G = G_0(i)
    print(i, is_triangle_free(G), is_P19_free(G))

0 True True
1 True True
2 True True
3 True True
4 True True


In [8]:
def G_1():
    # constructs a graph G_1 for analyzing induced paths with one x-type vertex v and one b-type vertex
    # V - number from 0 to 5, for different types of x-[gadget with c] connection
    G = raw_construction(1)
    for C in (0,1):
        for k in range(3):
            add_ac_pair(G, C, k, 0) # we attach one ac-pair of each type to v
    
    add_gadget(G, 1, [], 0)
    add_gadget(G, 1, [], 1) # both gadgets are connected to v only by b-type vertices
                
    for _ in range(4):
        for C in (0,1):
            for k in range(3):
                add_ac_pair(G, C, k) # we attach four more sets of ac-pairs
    return G

In [9]:
G = G_1()
print(is_triangle_free(G), is_P19_free(G))
#for V in range(6):
#    G = one_x_one_b(V)
#    print(V, is_triangle_free(G), is_P19_free(G))

True True


In [10]:
def G_2():
    # constructs a graph G_2 for analyzing induced paths with one x-type vertex v and two b-type vertices
    G = raw_construction(1)
    
    N = len(G)
    add_gadget(G, 1, [], 0) # by default, this gadget will be connected to v by b-type vertices
    add_gadget(G, 1, [], 0) # by default, this gadget will be connected to v by b-type vertices
        
    N = len(G)
    add_gadget(G, 1, [], 1) # by default, this gadget will be connected to v by b-type vertices
    add_gadget(G, 1, [], 1) # by default, this gadget will be connected to v by b-type vertices
                
    for _ in range(4):
        for C in (0,1):
            for k in range(3):
                add_ac_pair(G, C, k) # we attach four more sets of ac-pairs
    
    return G

In [11]:
G = G_1_2()
print(is_triangle_free(G), is_P19_free(G))

True True


In [12]:
def G_3():
    # constructs a graph G_3 for analyzing induced paths with two x-type vertices v, v'
    G = raw_construction(2)
    
    for _ in range(2):
        for C in (0,1):
            for k in range(3):
                add_ac_pair(G, C, k, 0)
                add_ac_pair(G, C, k, 1) # we attach two ac-pairs of each type to both v and v'
    
    for _ in range(4):
        for C in (0,1):
            for k in range(3):
                add_ac_pair(G, C, k) # we attach four more sets of ac-pairs

    b = G.add_vertex()
    G.add_edge(22, b)
    G.add_edge(23, b) # we attach v and v' to b-type vertex
    G.add_edge(b, get_Is()[1][0])
    G.add_edge(b, get_Is()[1][1]) # we attach b to I'               
    return G

In [13]:
G = G_2()
print(is_triangle_free(G), is_P19_free(G))

True True
